# 1. Imports and setup

In [34]:
#Import libraries

import matplotlib.pyplot as plt
import rasterio
from rasterio import plot
from rasterio.plot import show
from rasterio.windows import Window
from rasterio.transform import Affine
from PIL import Image, ImageDraw
import numpy as np
import os
import torch
import torch.nn as nn
import json
import xarray as xr
import rioxarray as rio
import cv2

torch.random.manual_seed(0)

In [2]:
!nvidia-smi

Thu May 15 15:55:53 2025       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 546.80                 Driver Version: 546.80       CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080 ...  WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   60C    P8              13W / 120W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

# 2. Convertion from files to desired input files

## 2.1 Json to binary mask

In [36]:
inputDir = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\Img\\MasksToPNGMasks"
mainOutput = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\Img\\MasksOutput"

In [38]:
def json_to_mask(json_path, output_dir):
    with open(json_path, 'r') as f:
        data = json.load(f)

    image_width = data.get('imageWidth')
    image_height = data.get('imageHeight')
    if image_width is None or image_height is None:
        raise ValueError("Las dimensiones de la imagen no están especificadas en el archivo JSON.")

    mask = Image.new('L', (image_width, image_height), 0)
    draw = ImageDraw.Draw(mask)

    for shape in data['shapes']:
        if (shape['label'] == "cropArea") or (shape['label'] == "CropArea"):
            points = shape['points']
            points_int = [(int(x), int(y)) for x, y in points]
            draw.polygon(points_int, outline=255, fill=255)

    # Obtener el nombre base del archivo JSON sin extensión
    base_name = os.path.splitext(os.path.basename(json_path))[0]
    # Crear el nombre del archivo de salida con el sufijo "_m.png"
    output_filename = f"{base_name}_m.png"
    # Unir la ruta del directorio de salida con el nombre del archivo
    output_path = os.path.join(output_dir, output_filename)

    # Guardar la máscara como imagen PNG
    mask.save(output_path)

In [39]:
# Loop through all .tif files in input_dir
for file in os.listdir(inputDir):
    jsonPath = os.path.join(inputDir, file)
    # Call the custom function to generate tiles and PNGs
    json_to_mask(jsonPath, mainOutput)

## 2.2 Process files from AI4boundaries Dataset

From nc in date 6 (early Northern Hemisphere autumn), to single image

In [68]:
inputTestDS = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\AI4boundaries\\test\\TestToProcess"
OutputTestDS = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\AI4boundaries\\test\\TestProcessed"

In [69]:
#Solo se usan las imagenes de AT, NL y SI de AI4boundaries

def procesar_nc_a_tif(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    for archivo in os.listdir(input_dir):
        if archivo.endswith(".nc"):
            ruta_nc = os.path.join(input_dir, archivo)
            nombre_base = os.path.splitext(archivo)[0]
            ruta_tif = os.path.join(output_dir, f"{nombre_base}.tif")

            ds = xr.open_dataset(ruta_nc, decode_coords="all")

            # Seleccionar la primera fecha
            ds_fecha0 = ds.isel(time=5)

            # Establecer las dimensiones espaciales
            ds_fecha0 = ds_fecha0.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=False)

            # Verificar si el CRS está presente
            if ds_fecha0.rio.crs is None:
                print(f"Advertencia: CRS no definido en {archivo}. Estableciendo CRS manualmente.")
                ds_fecha0 = ds_fecha0.rio.write_crs("EPSG:4326", inplace=False)

            # Eliminar el atributo 'grid_mapping' si existe
            for var in ds_fecha0.data_vars:
                if 'grid_mapping' in ds_fecha0[var].attrs:
                    del ds_fecha0[var].attrs['grid_mapping']

            # Seleccionar las bandas deseadas
            bandas = ['B4', 'B3', 'B2', 'B8', 'NDVI']  # Rojo, Verde, Azul, NIR, NDVI
            bandas_disponibles = [banda for banda in bandas if banda in ds_fecha0.variables]

            if not bandas_disponibles:
                print(f"No se encontraron las bandas especificadas en {archivo}.")
                continue

            # Crear una lista de DataArrays para las bandas seleccionadas
            dataarrays = []
            for banda in bandas_disponibles:
                da = ds_fecha0[banda]
                dataarrays.append(da)

            # Combinar las bandas en un solo DataArray multibanda
            da_combinado = xr.concat(dataarrays, dim="band")

            # Guardar como GeoTIFF
            da_combinado.rio.to_raster(ruta_tif)


In [ ]:
procesar_nc_a_tif(inputTestDS, OutputTestDS)

De imagen en tamaño 256 por 256, a 512 por 512

In [71]:
# 256 a 512
from skimage.transform import resize
from skimage.util import img_as_uint

def escalar_tiff_uint16(carpeta_entrada, carpeta_salida):
    os.makedirs(carpeta_salida, exist_ok=True)

    for nombre_archivo in os.listdir(carpeta_entrada):
        if nombre_archivo.lower().endswith((".tif", ".tiff")):
            ruta_entrada = os.path.join(carpeta_entrada, nombre_archivo)
            nombre_base = os.path.splitext(nombre_archivo)[0]
            ruta_salida = os.path.join(carpeta_salida, f"{nombre_base}_512.tif")

            with rasterio.open(ruta_entrada) as src:
                # Leer todas las bandas
                bandas = src.read()
                num_bandas, altura, anchura = bandas.shape
                
                # Redimensionar cada banda
                bandas_redimensionadas = np.empty((num_bandas, 512, 512), dtype=np.uint16)
                for i in range(num_bandas):
                    banda = bandas[i]
                    banda = banda.astype(np.float32)
                    banda = np.nan_to_num(banda, nan=0.0, posinf=0.0, neginf=0.0)

                    # Reemplazar valores < 0 por 0
                    banda[banda < 0] = 0

                    # Normalizar la banda a rango [0, 1] para resize
                    banda_normalizada = banda / 65535.0
                    banda_redimensionada = resize(
                        banda_normalizada,
                        (512, 512),
                        mode='reflect',
                        anti_aliasing=True
                    )
                    # Convertir de nuevo a uint16
                    bandas_redimensionadas[i] = img_as_uint(banda_redimensionada)

                # Crear perfil para el nuevo archivo
                perfil = src.profile
                perfil.update({
                    'driver': 'GTiff',
                    'count': num_bandas,
                    'dtype': 'uint16',
                    'width': 512,
                    'height': 512,
                    'nodata': 0,
                    'transform': rasterio.transform.from_origin(
                        src.transform.c,  # x origin
                        src.transform.f,  # y origin
                        src.transform.a * (anchura / 512),  # pixel width
                        src.transform.e * (altura / 512)   # pixel height
                    )
                })

                # Guardar la imagen redimensionada
                with rasterio.open(ruta_salida, 'w', **perfil) as dst:
                    dst.write(bandas_redimensionadas)


In [72]:
carpeta_entrada = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\AI4boundaries\\test\\TestProcessed"
carpeta_salida = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\AI4boundaries\\test\\Test256To512"
escalar_tiff_uint16(carpeta_entrada, carpeta_salida)

Máscara binaria para archivos tiff en las máscaras (tambien, otro reescalado previo, imputando valores bajos y NaNs)

In [83]:
def convertir_tiff_a_mascara_binaria(carpeta_entrada, carpeta_salida, umbral=127):
    os.makedirs(carpeta_salida, exist_ok=True)

    for nombre in os.listdir(carpeta_entrada):
        if nombre.lower().endswith((".tif", ".tiff")):
            ruta_tif = os.path.join(carpeta_entrada, nombre)
            nombre_salida = os.path.splitext(nombre)[0] + "_m.png"
            ruta_png = os.path.join(carpeta_salida, nombre_salida)

            with rasterio.open(ruta_tif) as src:
                banda = src.read(1)  # Solo primera banda

                # Convertir en máscara binaria: valores > umbral a blanco (255), el resto negro (0)
                mascara = np.where(banda > umbral, 255, 0).astype(np.uint8)

                # Guardar como PNG
                Image.fromarray(mascara).save(ruta_png)


In [73]:
carpeta_entrada = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\AI4boundaries\\masks\\TestMasksToProcess"
carpeta_salida1 = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\AI4boundaries\\masks\\TestMasks256To512"
carpeta_salida2 = "C:\\Users\\Leonardo\\Documents\\Tesis\\Tesis2\\Imagenes\\AI4boundaries\\masks\\TestMasksProcessed"

In [77]:
def escalar_tiff_uint16_2(carpeta_entrada, carpeta_salida):
    os.makedirs(carpeta_salida, exist_ok=True)

    for nombre_archivo in os.listdir(carpeta_entrada):
        if nombre_archivo.lower().endswith((".tif", ".tiff")):
            ruta_entrada = os.path.join(carpeta_entrada, nombre_archivo)
            nombre_base = os.path.splitext(nombre_archivo)[0]
            ruta_salida = os.path.join(carpeta_salida, f"{nombre_base}_512.tif")

            with rasterio.open(ruta_entrada) as src:
                bandas = src.read()
                num_bandas, altura, anchura = bandas.shape
                
                bandas_redimensionadas = np.empty((num_bandas, 512, 512), dtype=np.uint16)

                for i in range(num_bandas):
                    banda = bandas[i].astype(np.float32)

                    # Imputar 0 en NaN o valores vacíos
                    banda = np.nan_to_num(banda, nan=0.0)

                    # Recorte entre percentiles 2 y 98
                    p2, p98 = np.percentile(banda, (2, 98))
                    banda = np.clip(banda, p2, p98)

                    # Normalizar entre 0 y 1
                    rango = p98 - p2
                    if rango == 0:
                        banda_norm = np.zeros_like(banda)
                    else:
                        banda_norm = (banda - p2) / rango

                    # Redimensionar
                    banda_redim = resize(
                        banda_norm,
                        (512, 512),
                        mode='reflect',
                        anti_aliasing=True
                    )

                    # Convertir a uint16
                    banda_uint16 = np.clip(banda_redim * 65535, 0, 65535).astype(np.uint16)
                    bandas_redimensionadas[i] = banda_uint16

                perfil = src.profile
                perfil.update({
                    'driver': 'GTiff',
                    'count': num_bandas,
                    'dtype': 'uint16',
                    'width': 512,
                    'height': 512,
                    'nodata': 0,
                    'transform': rasterio.transform.from_origin(
                        src.transform.c,
                        src.transform.f,
                        src.transform.a * (anchura / 512),
                        src.transform.e * (altura / 512)
                    )
                })

                with rasterio.open(ruta_salida, 'w', **perfil) as dst:
                    dst.write(bandas_redimensionadas)

In [78]:
escalar_tiff_uint16_2(carpeta_entrada, carpeta_salida1)

In [84]:
convertir_tiff_a_mascara_binaria(carpeta_salida1, carpeta_salida2, umbral=0)